In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse

def get_filename(url: str):
    try:
        with requests.get(url) as req:
            if content_disposition := req.headers.get("Content-Disposition"):
                param, options = werkzeug.http.parse_options_header(content_disposition)
                if param == 'attachment' and (filename := options.get('filename')):
                    return filename

            path = urlparse(req.url).path
            name = path[path.rfind('/') + 1:]
            return name
    except requests.exceptions.RequestException as e:
        raise e
        
page_now = "https://www.cisa.gov/news-events/cybersecurity-advisories?f%5B0%5D=advisory_type%3A65"

soup = BeautifulSoup(requests.get(page_now).content, "html.parser")

index = []
titles = soup.find_all("h3",class_="c-teaser__title")
print("Page 0")
for title in titles:
    ref_link = title.find_all("a",href=True)
    if len(ref_link) > 0:
        article_name = ref_link[0].text.strip()
        article_url = "https://www.cisa.gov" + ref_link[0]['href']
        print(article_name)
        
        soup_report = BeautifulSoup(requests.get(article_url).content, "html.parser")
        
        downloads = soup_report.find_all("div",class_="c-file__download")
        
        for download in downloads:
            down_link = download.find_all("a",class_="c-file__link",href=True)
            file_name = down_link[0].text
            file_url = "https://www.cisa.gov" + down_link[0]['href']
            
            info = {"name":article_name,"url":article_url,"file_name":file_name,"file_url":file_url}
            index.append(info)
            
            r = requests.get(file_url, allow_redirects=True)
            
            original_name = get_filename(file_url)
            
            open(original_name, 'wb').write(r.content)

            
    else: continue
    
info_df = pd.DataFrame(index)
info_df.to_csv("cisa_reports.csv",index=False)

Page 0
MAR-10448362-1.v1 Volt Typhoon
MAR-10478915-1.v1 Citrix Bleed
MAR-10430311-1.v1 Multiple Nation-State Threat Actors Exploit CVE-2022-47966 and CVE-2022-42475
MAR-10454006.r5.v1 SUBMARINE, SKIPJACK, SEASPRAY, WHIRLPOOL, and SALTWATER Backdoors
Infamous Chisel Malware Analysis Report
MAR-10459736.r1.v1 WHIRLPOOL Backdoor
MAR-10454006.r4.v2 SEASPY and WHIRLPOOL Backdoors
MAR-10454006-r1.v2 SUBMARINE Backdoor
MAR-10454006-r2.v1 SEASPY Backdoor


In [ ]:
info_df.info()